In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning, LinAlgError
from scipy.optimize._optimize import OptimizeWarning
from scipy.optimize._linprog_util import _clean_inputs, _get_Abc
import warnings
import numpy as np
from functools import partial
from collections import namedtuple

from generate_instances import generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location

In [3]:
rng = np.random.RandomState(1)

In [33]:
root = './mid_setc'

In [34]:
!mkdir ./mid_setc
!mkdir ./mid_setc/raw
!mkdir ./mid_setc/processed

### Setcover

In [45]:
# density = 0.15
# nrows_l = 15
# nrows_u = 20
# ncols_l = 15
# ncols_u = 20

density = 0.01
nrows_l = 200
nrows_u = 250
ncols_l = 150
ncols_u = 200

bounds = (0., 1.)

surrogate_gen = partial(generate_setcover, nrows_l=nrows_l, nrows_u=nrows_u, ncols_l=ncols_l, ncols_u=ncols_u, density=density, rng=rng)

### Indset

In [6]:
def surrogate_gen():
    # nnodes = rng.randint(10, 20)
    nnodes = rng.randint(100, 150)
    # nnodes = rng.randint(300, 500)
    graph = Graph.barabasi_albert(number_of_nodes=nnodes, affinity=2, random=rng)
    A, b, c = generate_indset(graph=graph, nnodes=nnodes)
    return A, b, c

bounds = (0., 1.)

### Cauctions

In [ ]:
def surrogate_gen():
    n_items=rng.randint(20, 21)
    n_bids=rng.randint(20, 21)
    # n_items = 500
    # n_bids = 600
    A, b, c = generate_cauctions(n_items=n_items, n_bids=n_bids, rng=rng, min_value=0.01, max_value=0.1)
    return A, b, c

bounds = None

### Facilities

In [14]:
def surrogate_gen():
    # n_customers = rng.randint(3, 5)
    # n_facilities = rng.randint(3, 5)
    n_customers = rng.randint(10, 15)
    n_facilities = rng.randint(10, 15)
    ratio = 5
    A, b, c = generate_capacited_facility_location(n_customers=n_customers, n_facilities=n_facilities, ratio=ratio, rng=rng)
    return A, b, c

bounds = (0., 1.)

## create

In [17]:
_LPProblem = namedtuple('_LPProblem',
                        'c A_ub b_ub A_eq b_eq bounds x0 integrality')
_LPProblem.__new__.__defaults__ = (None,) * 7  # make c the only required arg

# eq

In [18]:
from solver.linprog_ip import _ip_hsd_feas
from scipy.linalg import null_space
from torch_geometric.data import Batch, HeteroData, InMemoryDataset

In [51]:
warnings.filterwarnings("error")

ips = []
graphs = []
pkg_idx = 0
success_cnt = 0
fail_cnt = 0

max_iter = 12000
num = 10000

for i in tqdm(range(max_iter)):
    A, b, c = surrogate_gen()
    c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
    Ab = np.concatenate([A, b[:, None]], axis=1)
    max_logit = np.abs(Ab).max(axis=1)
    max_logit[max_logit == 0] = 1.
    Ab = Ab / max_logit[:, None]
    A = Ab[:, :-1]
    b = Ab[:, -1]

    # process LP into standard form Ax=b, x>=0
    lp = _LPProblem(c, A, b, None, None, bounds, None, None)
    lp = _clean_inputs(lp)
    A, b, c, *_ = _get_Abc(lp, 0.)
    
    try:
        res = linprog(
                c, 
                A_ub=None,
                b_ub=None,
                A_eq=A, b_eq=b, bounds=None, method='interior-point')
    except (LinAlgWarning, OptimizeWarning, AssertionError, LinAlgError):
        fail_cnt += 1
        continue
    else:
        if res.success and not np.isnan(res.fun):
            nulls = null_space(A)
            proj = nulls @ (nulls / (nulls * nulls).sum(0)).T

            ips.append((A.astype(np.float32),
                        b.astype(np.float32),
                        c.astype(np.float32),
                        res.x,
                        proj))

            # create graph on the fly
            x_feasible, *_ = _ip_hsd_feas(A, b, np.zeros(A.shape[1]), 0.,
                                          alpha0=0.99995, beta=0.1,
                                          maxiter=100, tol=1.e-6, sparse=False,
                                          lstsq=False, sym_pos=True, cholesky=None,
                                          pc=True, ip=True, permc_spec='MMD_AT_PLUS_A',
                                          rand_start=True)
            
            A = torch.from_numpy(A).to(torch.float)
            b = torch.from_numpy(b).to(torch.float)
            c = torch.from_numpy(c).to(torch.float)
            x = torch.from_numpy(res.x).to(torch.float)
            x_feasible = torch.from_numpy(x_feasible).to(torch.float)
            
            A_row, A_col = torch.where(A)
            
            # proj_matrix @ x projects onto the nullspace of A
            proj_matrix = torch.from_numpy(proj).to(torch.float)
            
            data = HeteroData(
                cons={'x': torch.cat([A.mean(1, keepdims=True),
                                      A.std(1, keepdims=True)], dim=1)},
                vals={'x': torch.cat([A.mean(0, keepdims=True),
                                      A.std(0, keepdims=True)], dim=0).T},
                obj={'x': torch.cat([c.mean(0, keepdims=True),
                                     c.std(0, keepdims=True)], dim=0)[None]},
            
                cons__to__vals={'edge_index': torch.vstack(torch.where(A)),
                                'edge_attr': A[torch.where(A)][:, None]},
                vals__to__obj={'edge_index': torch.vstack([torch.arange(A.shape[1]),
                                                           torch.zeros(A.shape[1], dtype=torch.long)]),
                               'edge_attr': c[:, None]},
                cons__to__obj={'edge_index': torch.vstack([torch.arange(A.shape[0]),
                                                           torch.zeros(A.shape[0], dtype=torch.long)]),
                               'edge_attr': b[:, None]},
                x_solution=x,
                x_feasible=x_feasible,
                obj_solution=c.dot(x),
                c=c,
                b=b,
                A_row=A_row,
                A_col=A_col,
                A_val=A[A_row, A_col],
                proj_matrix=proj_matrix.reshape(-1),
            )
            success_cnt += 1
            graphs.append(data)

    if len(ips) >= 1000 or success_cnt == num:
        # with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
        #     pickle.dump(ips, file)
        torch.save(Batch.from_data_list(graphs), f'{root}/processed/batch{pkg_idx}.pt')
        pkg_idx += 1
        graphs = []
        ips = []

    if success_cnt >= num:
        break

warnings.resetwarnings()

 63%|██████████████████████████████████████████████████████████████████████████████▎                                              | 7523/12000 [3:16:43<1:57:04,  1.57s/it]
